# NAS IMPACT DATABASE SCRAPER
This code scrapes impact data from the Nonindingenous Aquatic Species (https://nas.er.usgs.gov/) database. Rather than connecting from backend, this pulls from the frontend of the data entry portal.


In [2]:
# import libraries
import pandas as pd                                           # handle dataframes
import openpyxl                                               # import and export Excel files
from bs4 import BeautifulSoup                                 # read HTML
import time                                                   # set timers
import re                                                     # clean text data
from datetime import date                                     # get date to organize output files
from rpy2.robjects.packages import importr                    # import packages from R - needed for taxize package
from selenium import webdriver                                # automate web browser interaction
from selenium.webdriver.common.keys import Keys               # automate keyboard actions
from selenium.webdriver.common.by import By                   # find elements by html id on webpage
from selenium.webdriver.support.ui import Select              # automate dropdown selection
from selenium.common.exceptions import WebDriverException     # deal with exptions in webdriver


### Open Chrome Web Driver
Opens to NAS sign in page. Manually enter personal login information

In [5]:
# start web driver - use login information
driver = webdriver.Chrome()
driver.get('https://nas.er.usgs.gov/Signin.aspx')


### Pull NAS Impact Data
This scrapes NAS impact data in two parts. It first scapes the tables on the NAS impacts list page. Unfortunantly, data from tables is incomplete. The second step scrapes the full impact description statement, notes, and geographic location from each impact ID page.

In [7]:
# set timer
time_start = time.time()

# function to extract table from HTML
def find_table():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    data_table = soup.find('table', id='ContentPlaceHolder1_gvImpacts')
    return data_table

# function to extract and append table data
def extract_table_data(table):
    for row in table.find_all('tr')[3:-2]:
        row_data = []
        for index, cell in enumerate(row.find_all('td')[1:-1]):
            if index == 8:
                checkmark_image = cell.find('img')
                if checkmark_image:
                    row_data.append('yes')
                else:
                    row_data.append('')
            else:
                row_data.append(cell.get_text().strip())
        data.append(row_data)
    return data

# create empty list to store table data
data = []

# create list - this will allow code to click through impact database table pages
# n is based on number of pages and will likely need to be manually updated - consider creating code in future
n = 75
table_clicks = list(range(2, 12)) + (list(range(4, 14)) * n)

# extract table headers
driver.get('https://nas.er.usgs.gov/DataEntry/Impact/ImpactsList.aspx')
table = find_table()
header = [th.text.strip() for th in table.find_all('th')[1:-1]]

# extract data from first table
extract_table_data(table)

# extract data from remaining tables
try:
    for i in table_clicks:
        element = f'/html/body/form/div[4]/div/table/tbody/tr[1]/td/table/tbody/tr/td[{i}]/a'
        link = driver.find_element(By.XPATH, element)
        link.click()
        table = find_table()
        extract_table_data(table)
except:
    print(f"An error occurred:{e}")
    pass

# create data table    
impact_data = pd.DataFrame(data, columns = header)

# remove any potiential duplicates based on Impact ID
impact_data.drop_duplicates(subset='Impact ID', keep='first', inplace=True)


# !! not all data appears completely in data tables - need to going into edit option
      
# create new column for Geographic Location
impact_data['Geographic Location'] = None

# go through each individual impact to get full Impact Description, Notes, and Geographic Location
for index, row in impact_data.iterrows():
    try:
        url = 'https://nas.er.usgs.gov/DataEntry/Impact/Impacts.aspx?ImpactID=' + str(row['Impact ID'])
        driver.get(url)
        impact_description = driver.find_element(By.XPATH, '/html/body/form/div[4]/div/div/table[2]/tbody/tr[6]/td[2]/textarea' )
        impact_data.at[index, 'Impact Description'] = impact_description.get_attribute('value')
        notes = driver.find_element(By.XPATH, '/html/body/form/div[4]/div/div/table[2]/tbody/tr[7]/td[2]/textarea')
        impact_data.at[index, 'Notes'] = notes.get_attribute('value')
        geographic_loc = driver.find_element(By.XPATH, '/html/body/form/div[4]/div/div/table[2]/tbody/tr[9]/td[2]/input')
        impact_data.at[index, 'Geographic Location'] = geographic_loc.get_attribute('value')
    except:
        driver.refresh()
        index -= 1
        continue

print(f'Impact Info Time: {round((time.time()- time_start)/60)}')

# create an output file with all impacts - this ensures database does not need to be scraped again if 
# work needs to be done later on    
impact_data.to_excel('output_' + str(date.today()) + '.xlsx', index = False)


Impact Info Time: 64


### Edit NAS Impact Data
Scientific and common names are added to NAS impact data. Furthermore, the TSN text strings are split apart and separated in individual rows. This improves GLANSIS teams ability to find which invasives are affecting individual impacted species. Text and columns are cleaned and rearranged for easier reading and viewing. 

If you get this error: 

"Error:
! The `x` argument of `as_tibble()` can't be missing as of tibble 3.0.0.
Run `rlang::last_error()` to see where the error occurred."

The ITIS website is likely down. Try again later.

In [3]:
# This piece of code uploads archived version of impact database
# impact_data = pd.read_excel(r'output_.xlsx', dtype = str)

In [4]:
# create a species list with all species in NAS database
nas_species_list = pd.read_excel(r'speciesListNAS.xlsx', dtype = str)
nas_species_list = nas_species_list[['species.id', 'scientific.name', 'common.name', 'native.exotic']]

# merge species list with NAS impacts to add species names
merged_impact_data = pd.merge(impact_data, nas_species_list, left_on='Species ID', right_on='species.id', how='left')

# convert text strings in 'Associated TSNs' column into a list
merged_impact_data['Associated TSNs'] = merged_impact_data['Associated TSNs'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# seperate each entry in 'Associated TSNs' column into its own row
explode_data = merged_impact_data.explode('Associated TSNs')

# the following may require R and taxize package to be installed 
# import taxize package from R
taxize = importr("taxize")

# function to get scientific name from ITIS number
def get_species(itis_number):
    if pd.isna(itis_number):
        return ''
    else:     
        try:
            name = str(taxize.id2name(itis_number, db='itis')[0].rx2('name')).split('"')[1]
            return name
        except Exception as e:
            print(f"An error occurred while retrieving species name for ITIS number {itis_number}: {e}")
            return 'Error'

# apply custum get_species function to 'Associated TSNs'
explode_data['Impacted Species'] = explode_data['Associated TSNs'].apply(get_species)

# rename columns
explode_data = explode_data.rename(columns={'scientific.name': 'Scientific Name', 'common.name': 'Common Name', 'native.exotic':'Status'})

# reorder and select columns
NAS_impact_data = explode_data[['Impact ID', 'Species ID', 'Scientific Name', 'Common Name', 'Status', 'Impact Type', 'Impact Description', 'Study Type', 'Study Location', 'Reference Number', 'Associated TSNs', 'Impacted Species', 'Great Lakes Region', 'Geographic Location', 'Notes']]

# remove <em> hypertext - makes easier to read impact descriptions
NAS_impact_data.loc[:,'Impact Description'] = NAS_impact_data['Impact Description'].str.replace('<em>', '').str.replace('</em>', '')

# remove line breaks
NAS_impact_data.loc[:, 'Impact Description'] = NAS_impact_data['Impact Description'].str.replace('\n', '')

# export to Excel
NAS_impact_data.to_excel('NAS_output_' + str(date.today()) + '.xlsx', index = False)


### Clean GLANSIS Impact Data
Subset the GLANSIS species from the NAS species. Drop and replace the status column because GLANSIS uses different terms for invasive species.

In [8]:
# read in GLANSIS species list
glansis_species_list = pd.read_excel(r'speciesListGLANSIS.xlsx', dtype = str)

# create a list of species id's to subset NAS dataset
glansis_subset_list = list(glansis_species_list['species.id'])

# subset NAS dataset to get only GLANSIS species impacts
subset_impact_data = NAS_impact_data[NAS_impact_data['Species ID'].isin(glansis_subset_list)]

# drop NAS Status column - GLANSIS uses slightly different status terminology
subset_impact_data.drop(columns = ['Status'], inplace = True)

# merge glansis_species_list with subset_impact_data to get GLANSIS status and group
merged_impact_data = pd.merge(subset_impact_data, glansis_species_list, left_on='Species ID', right_on='species.id', how='left')

# rename columns
merged_impact_data = merged_impact_data.rename(columns={'status':'Status', 'group':'Group'})

# reorder and select columns
GLANSIS_impact_data = merged_impact_data[['Impact ID', 'Species ID', 'Scientific Name', 'Common Name', 'Group', 'Status', 'Impact Type', 'Impact Description', 'Study Type', 'Study Location', 'Reference Number', 'Associated TSNs', 'Impacted Species', 'Great Lakes Region', 'Geographic Location', 'Notes']]

# export GLANSIS impact data to Excel
GLANSIS_impact_data.to_excel('GLANSIS_output_' + str(date.today()) + '.xlsx', index = False)


C:\Users\redinger\AppData\Local\Temp\1\ipykernel_27412\1641211976.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_impact_data.drop(columns = ['Status'], inplace = True)


### *Coregonus Artedi* selection for panal review
This pulls are impacts related to cisco. This can be modified for other species reviews.

In [32]:
# create mask on TSN numbers
itis_mask = GLANSIS_impact_data['Associated TSNs'].str.contains(r'\b623384\b', case = False)

# create masks on Impact Descriptions
science_name_mask = GLANSIS_impact_data['Impact Description'].str.contains(r'\bCoregonus artedi\b', case = False)
abbrev_mask = GLANSIS_impact_data['Impact Description'].str.contains(r'\bC. artedi\b', case = False)
cisco_mask = GLANSIS_impact_data['Impact Description'].str.contains(r'\bcisco\b', case = False)
lakeherring_mask = GLANSIS_impact_data['Impact Description'].str.contains(r'\blake herring\b', case = False)

# apply masks to GLANSIS data
masked_data = GLANSIS_impact_data[cisco_mask | science_name_mask | abbrev_mask | lakeherring_mask | itis_mask ]

# handle duplications - this removed duplicated from 'exploded' data

# export cisco data to excel
cisco_data.to_excel('cisco_impact_data_' + str(date.today()) + '.xlsx', index = False)


In [24]:
# Sample dataframe
data = {
    'Impact_ID': [1, 2, 3, 2, 5, 3, 7],
    'Species': ['Coregonus albus', 'Salmo trutta', 'Salmo trutta', 'Salvelinus fontinalis', 'Salmo salar', 'Coregonus lavaretus', 'Coregonus oxyrhynchus']
}

df = pd.DataFrame(data)

# Function to filter duplicates based on conditions
def filter_duplicates(df):
    # Identify duplicates
    duplicates = df[df.duplicated(subset='Impact_ID', keep=False)]
    
    # Identify duplicates that are not Coregonus species
    non_coregonus_duplicates = duplicates[~duplicates['Species'].str.contains('Coregonus')]
    
    # Get indexes of duplicates that need to be removed
    indexes_to_remove = non_coregonus_duplicates.index
    
    # Filter out rows with indexes to remove
    filtered_df = df.drop(indexes_to_remove)
    
    return filtered_df

# Apply function
filtered_df = filter_duplicates(df)

print(filtered_df)

   Impact_ID                Species
0          1        Coregonus albus
4          5            Salmo salar
5          3    Coregonus lavaretus
6          7  Coregonus oxyrhynchus


In [25]:
df

,Impact_ID,Species
0,1,Coregonus albus
1,2,Salmo trutta
2,3,Salmo trutta
3,2,Salvelinus fontinalis
4,5,Salmo salar
5,3,Coregonus lavaretus
6,7,Coregonus oxyrhynchus


In [28]:
data = {
    'Impact_ID': [1, 2, 3, 2, 5, 3, 7],
    'Species': ['Coregonus albus', 'Salmo trutta', 'Salmo trutta', 'Salvelinus fontinalis', 'Salmo salar', 'Coregonus lavaretus', 'Coregonus oxyrhynchus']
}

df = pd.DataFrame(data)

# Create a mask to identify rows with 'Coregonus'
coregonus_mask = df['Species'].str.contains('Coregonus')

# Create a mask to identify duplicate Impact_ID
duplicate_mask = df.duplicated(subset='Impact_ID', keep=False)

# Create a mask to identify the first duplicate row without 'Coregonus'
first_duplicate_non_coregonus_mask = df[duplicate_mask & ~coregonus_mask].duplicated(subset='Impact_ID', keep='first')

# Apply conditions to filter the dataframe
filtered_df = df[~((duplicate_mask & ~coregonus_mask) & ~first_duplicate_non_coregonus_mask) | coregonus_mask]

print(filtered_df)

   Impact_ID                Species
0          1        Coregonus albus
3          2  Salvelinus fontinalis
4          5            Salmo salar
5          3    Coregonus lavaretus
6          7  Coregonus oxyrhynchus
